In [ ]:
import xarray as xr
import os
import numpy as np
import pandas as pd

from functools import partial

GRAPH_LT_DATA_PATH = "/g/data/wa46/user/nl5316/tw_spatial/graphcast_with_lead_time/"

In [ ]:
file_list = sorted([f for f in os.listdir(GRAPH_LT_DATA_PATH) if f.endswith(".nc")])
ncs = file_list
ncs = [GRAPH_LT_DATA_PATH + "/" + nc for nc in ncs]

In [ ]:
broadcast_array = xr.DataArray(
    [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    dims=["lead_time"],
    coords={"lead_time": pd.timedelta_range(start="6h", end="2d", freq="6h").values},
)


def broadcast(x, broadcast_array):
    x = x.broadcast_like(broadcast_array)
    return x


partial_func = partial(broadcast, broadcast_array=broadcast_array)

ds = xr.open_mfdataset(
    ncs, combine="nested", concat_dim="time", preprocess=partial_func
)

In [ ]:
ds = ds.dropna("lead_time", how="all")

In [ ]:
da = ds.apcp.compute()

In [ ]:
da.to_netcdf("/g/data/wa46/user/nl5316/tw_spatial/graphcast_combined/graphcast.nc")